In [1]:
!pip install catboost

  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/1c/e1/78e635a1e5f0066bd02a1ecfd658ad09fe30d275c65c2d0dd76fe253e648/catboost-1.2.7-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for graphviz from https://files.pythonhosted.org/packages/00/be/d59db2d1d52697c6adc9eacaf50e8965b6345cc143f671e1ed068818d5cf/graphviz-0.20.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB 660.6 kB/s eta 0:02:34
   ---------------------------------------- 0.0/101.7 MB 326.8 kB/s eta 0:05:12
   ---------------------------------------- 0.1/101.7 MB 653.6 kB/s eta 0:02:36
   ---------------------------------------- 0.1/101.7 MB 726.2 kB/s eta 0:02:20
   ---------------------------------------- 0.2/101.7 MB 794.9 kB/s eta 0:02:08
   ---------------------------------------- 0.3/101.7 MB 1.0 MB/s eta 0:01:40
   ------------------------------------

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [3]:
train_dir = r'D:\\my project\\fourth year\\computer vision\\dataset\\train'  
test_dir = r'D:\\my project\\fourth year\\computer vision\\dataset\\test'

In [4]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [5]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [6]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 4010 images belonging to 2 classes.


In [7]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 3543 images belonging to 2 classes.


In [8]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\mohamed\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

C:\Users\mohamed\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


126/126 ━━━━━━━━━━━━━━━━━━━━ 22s 172ms/step


In [10]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

 35/111 ━━━━━━━━━━━━━━━━━━━━ 10s 136ms/step

C:\Users\mohamed\anaconda\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


111/111 ━━━━━━━━━━━━━━━━━━━━ 37s 334ms/step


In [11]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 0.6660461	total: 144ms	remaining: 2m 24s
1:	learn: 0.6431720	total: 167ms	remaining: 1m 23s
2:	learn: 0.6264328	total: 193ms	remaining: 1m 4s
3:	learn: 0.6090638	total: 215ms	remaining: 53.6s
4:	learn: 0.5885011	total: 237ms	remaining: 47.2s
5:	learn: 0.5787905	total: 258ms	remaining: 42.8s
6:	learn: 0.5643714	total: 280ms	remaining: 39.8s
7:	learn: 0.5520049	total: 301ms	remaining: 37.3s
8:	learn: 0.5423681	total: 322ms	remaining: 35.4s
9:	learn: 0.5314610	total: 340ms	remaining: 33.7s
10:	learn: 0.5212884	total: 358ms	remaining: 32.2s
11:	learn: 0.5152682	total: 378ms	remaining: 31.1s
12:	learn: 0.5083899	total: 397ms	remaining: 30.2s
13:	learn: 0.5021393	total: 418ms	remaining: 29.5s
14:	learn: 0.4956831	total: 439ms	remaining: 28.8s
15:	learn: 0.4888991	total: 460ms	remaining: 28.3s
16:	learn: 0.4831389	total: 482ms	remaining: 27.8s
17:	learn: 0.4774023	total: 500ms	remaining: 27.3s
18:	learn: 0.4724408	total: 521ms	remaining: 26.9s
19:	learn: 0.4664964	total: 542ms	remai

In [12]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [13]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 63.05%


In [14]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [15]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [17]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model('cnn_feature_extractor6.h5')
catboost_classifier = joblib.load('catboost_classifier.pkl')  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'with_mask', 1: 'without_mask'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = r'D:\my project\fourth year\computer vision\dataset\test\with_mask\with_mask_2019.jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
The image is classified as: with_mask
